# S21 - GPT Training

In [1]:
!cd /kaggle/working

In [2]:
!rm -rf TSAI_ERA_Assignments
!git clone https://github.com/ToletiSri/TSAI_ERA_Assignments.git

Cloning into 'TSAI_ERA_Assignments'...
remote: Enumerating objects: 716, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 716 (delta 89), reused 178 (delta 75), pack-reused 512
Receiving objects: 100% (716/716), 24.85 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (365/365), done.


In [3]:
cd TSAI_ERA_Assignments/S21

/kaggle/working/TSAI_ERA_Assignments/S21


In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import config as cfg

In [5]:
device = cfg.device
block_size = cfg.block_size
batch_size = cfg.batch_size
eval_iters = cfg.eval_iters

In [6]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
from gpt import GPTLanguageModel

model = GPTLanguageModel(vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

for iter in range(cfg.max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % cfg.eval_interval == 0 or iter == cfg.max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")        

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
torch.save(model.state_dict(), 'saved_model.pth')


10.788929 M parameters
step 0: train loss 4.2221, val loss 4.2306
step 500: train loss 1.7587, val loss 1.9086
step 1000: train loss 1.3919, val loss 1.5998
step 1500: train loss 1.2673, val loss 1.5301
step 2000: train loss 1.1904, val loss 1.5142
step 2500: train loss 1.1194, val loss 1.5024
step 3000: train loss 1.0738, val loss 1.4949
step 3500: train loss 1.0213, val loss 1.5150
step 4000: train loss 0.9618, val loss 1.5183
step 4500: train loss 0.9087, val loss 1.5446
step 4999: train loss 0.8543, val loss 1.5633


In [11]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Officious, and the people of the fire;
And him with his virtuous natural words;
The sword of a parageit and confirmate
Where it had our courtesy: nor my garms should we,
For shame, but defy him-shamed with the gentle hours
That sweet's infant; and so retoris as it glow,
To help misforthy, every known, which shows
In commissingly strength; for near nature wears, each in
joy.
I am enten'd to no good fellow, health I wear:
Ah, tell-my mother may stone return to me;
And leave me have this country'd 
